<h2><left>Hello, This Notebook Contains example of Hand Written Digits Recognition Algorithm </left></h2>
<h4><I><left>- We recognize hand written digits based on scikit-learn library.</left></I></h4>
<h4><I><left>- We try to expand dataset based on openCV library, and retrain our models.</left></I></h4>
<div class="alert alert-block alert-info" style="margin-top: 20px">
        <ul>
            <li><h4>Contents:</h4></li>
    <ul>
        <li><a href="#load">Loading Dataset</a></li>
        <li><a href="#models">Models</a></li>
        <ul>
            <li><a href="#k_neighbors">KNeighbors</a></li>
            <li><a href="#multinomial">Multinomial</a></li>
        </ul>
        <li><a href="#expanding">Expanding Dataset</a></li>
    </ul>
    </ul>
</div>
<hr>

<a id="load"></a>
<h3><left>- Getting Data:</left></h3>
<h4><I><left>- loading mnist dataset, plotting some images from dataset.</left></I></h4>

In [ ]:
#import libs
from sklearn.datasets import fetch_openml
import numpy as np

In [2]:
#import data: 
mnist = fetch_openml('mnist_784',version=1)

In [3]:
X, y = mnist['data'],mnist['target']

In [4]:
import matplotlib.pyplot as plt

def show_some_image(data_sample):
    digit_image= data_sample.reshape(28,28)
    plt.imshow(digit_image,cmap='binary')
    plt.axis('off')
    plt.show()
    
def processed_labels(labels):
    lb= labels.astype(np.uint8)
    return lb

def train_test_split(X,y,split_value):
    X_train, X_test, y_train, y_test = X[:split_value], X[split_value:], y[:split_value], y[split_value:]
    return X_train,X_test,y_train,y_test

In [5]:
y = processed_labels(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,60000)

In [ ]:
show_some_image(X_test[1])

<a id="models"></a>
<h3><left>- Models:</left></h3>
<h4><I><left>- training KNeighborsClassifier, MultinomialNB and plotting score metrices.</left></I></h4>

<a id="k_neighbors"></a>
<h4><left>- KNeighbors:</left></h4>

In [6]:
# models:
from sklearn.neighbors import KNeighborsClassifier
n_clf = KNeighborsClassifier(weights='distance', n_neighbors=4)
n_clf.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [ ]:
# from sklearn.model_selection import cross_val_score
# y_predict = cross_val_score(n_clf,X_train,y_train,cv=3,scoring='accuracy')
# from sklearn.model_selection import cross_val_predict
# y_pred = cross_val_predict(c_clf,X_train,y_train,cv=2)

In [ ]:
n_clf.predict(X_test[:100])

In [7]:
X_test = np.array(X_test)

In [8]:
y_predict = n_clf.predict(X_test)

In [11]:
y_p = np.array([])
for i in range(10000):
    y_p=np.append(y_p,n_clf.predict([X_test[i]]))

In [27]:
from sklearn.metrics import accuracy_score

print(confusion_matrix(y_predict,y_test))
print(accuracy_score(y_test, y_predict))
print(precision_score(y_test,y_predict,average = 'weighted'))
print(recall_score(y_test,y_predict,average = 'weighted'))

[[ 973    0   10    0    1    4    4    0    5    3]
 [   1 1132    5    1    5    0    2   17    2    4]
 [   1    2  995    3    0    0    0    4    4    2]
 [   0    0    2  974    0    9    0    0   14    7]
 [   0    0    1    1  950    2    3    3    5    9]
 [   1    0    0   14    0  862    3    0   11    4]
 [   3    1    0    1    4    7  946    0    4    1]
 [   1    0   16    7    3    1    0  994    4   10]
 [   0    0    3    4    0    3    0    0  920    1]
 [   0    0    0    5   19    4    0   10    5  968]]
0.9714
0.9715166824529755
0.9714


<a id="multinomial"></a>
<h4><left>- MultinomialNB:</left></h4>

In [15]:
from sklearn.naive_bayes import MultinomialNB ,BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB()

In [19]:
predictions_mnb = mnb.predict(X_test)
confusion_matrix(predictions_mnb, y_test)

array([[ 912,    0,   15,    4,    2,   23,   17,    1,    6,    6],
       [   0, 1061,   11,   11,    2,   11,   13,   21,   26,    7],
       [   2,    5,  858,   34,    6,    6,   17,   11,   13,    3],
       [   6,    9,   24,  851,    0,  107,    1,    5,   54,   10],
       [   1,    0,   10,    1,  732,   18,    7,   19,   14,   66],
       [   8,    2,    3,   21,    0,  590,   25,    0,   27,   10],
       [  14,    6,   33,    7,   25,   17,  860,    1,    8,    0],
       [   1,    0,   11,   14,    1,    6,    0,  861,    9,   17],
       [  36,   51,   66,   40,   38,   78,   18,   40,  777,   27],
       [   0,    1,    1,   27,  176,   36,    0,   69,   40,  863]],
      dtype=int64)

In [24]:
print(accuracy_score(y_test,predictions_mnb))
print(precision_score(y_test,predictions_mnb,average = 'weighted'))
print(recall_score(y_test,predictions_mnb,average = 'weighted'))

0.8365
0.844464008745313
0.8365


<a id="expanding"></a>
<h3><left>- Expanding Dataset:</left></h3>
<h4><I><left>- we try to expand dataset and retrain our models again.</left></I></h4>

In [60]:
import cv2 as cv

X_train_expanded = np.array(X_train)
y_train_expanded = np.array(y_train)

def shift_image(data,dx,dy):
    M = np.float32([[1,0,dx],[0,1,dy]])  
    return cv.warpAffine(data,M,(data.reshape(28,28)).shape).reshape(784)
       
# print(X_train_expanded.shape)
# shifted_images = np.apply_along_axis(shift_image, axis=1, arr=X_train, dx=1, dy=0)
# print(shifted_images.shape)
# X_ = np.r_[X_train_expanded,shifted_images]
# X_.shape

In [61]:
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    shifted_images = np.apply_along_axis(shift_image, axis=1, arr=X_train, dx=dx, dy=dy)
    X_train_expanded = np.r_[X_train_expanded,shifted_images]
    y_train_expanded= np.r_[y_train_expanded,y_train]
    
X_train_expanded.shape, y_train_expanded.shape 

((300000, 784), (300000,))

In [67]:
mnb_expand = MultinomialNB()
mnb_expand.fit(X_train_expanded, y_train_expanded)

MultinomialNB()

In [68]:
y_perdict_= mnb_expand.predict(X_test)

In [69]:
print(accuracy_score(y_test,y_perdict_))
print(precision_score(y_test,y_perdict_,average = 'weighted'))
print(recall_score(y_test,y_perdict_,average = 'weighted'))

0.8365
0.844464008745313
0.8365


<h3><I><center>...The End... </center></I></h3>